# ETL (extracción, transformación y carga)

In [1]:
#librerias necesarias para el eda
import os
import pandas as pd

#definimos la carpeta de los datasets:
carpeta = 'datasets' #crear carpeta si no existe
if not os.path.exists(carpeta):
    os.makedirs(carpeta)

Convertimos datos de excel a csv para una facil manipulación

In [2]:
#excel_files = ['internet.xlsx'] #archivo obligatorio
excel_files = [
    'Internet.xlsx',
    'mapa_conectividad.xlsx',
    'Portabilidad.xlsx',
    #'servicios_postales.xlsx', #archivo complementario ignorado por que no aporta nada al objetivo del analisis
    'telefonia_fija.xlsx',
    #'telefonia_movil.xlsx',
    'Television.xlsx'
]   #los comentados son archivo complementarios ignorados por que no
    #aportan nada al objetivo del analisis

def excel_a_csv(archivo): #excel a csv
    xls = pd.ExcelFile(archivo)
    for sheet_name in xls.sheet_names:
        #leer hojas excel
        df = pd.read_excel(archivo, sheet_name=sheet_name)
        
        #definir name de csv
        csv_file_name = f"{os.path.splitext(os.path.basename(archivo))[0]}_{sheet_name}.csv"
        csv_file_path = os.path.join(carpeta, csv_file_name)
        
        #guardar csv con nombre corregido
        
        csv_file_path = csv_file_path.replace(' .csv', '.csv')
        csv_file_path = csv_file_path.replace(' ', '_')
        csv_file_path = csv_file_path.replace('á', 'a')
        csv_file_path = csv_file_path.replace('é', 'e')
        csv_file_path = csv_file_path.replace('í', 'i')
        csv_file_path = csv_file_path.replace('ó', 'o')
        csv_file_path = csv_file_path.replace('ú', 'u')
        
        df.to_csv(csv_file_path, index=False)

        print(f"Guardado: {csv_file_path}")

#bucle
for file in excel_files:
    excel_a_csv(file)

print("Conversión lista")

Guardado: datasets\Internet_Acc_vel_loc_sinrangos.csv
Guardado: datasets\Internet_Velocidad_sin_Rangos.csv
Guardado: datasets\Internet_Accesos_tecnologia_localidad.csv
Guardado: datasets\Internet_Velocidad_%_por_prov.csv
Guardado: datasets\Internet_Totales_VMD.csv
Guardado: datasets\Internet_Totales_Accesos_Por_Tecnologia.csv
Guardado: datasets\Internet_Accesos_Por_Tecnologia.csv
Guardado: datasets\Internet_Penetracion-poblacion.csv
Guardado: datasets\Internet_Penetracion-hogares.csv
Guardado: datasets\Internet_Penetracion-totales.csv
Guardado: datasets\Internet_Totales_Accesos_por_velocidad.csv
Guardado: datasets\Internet_Accesos_por_velocidad.csv
Guardado: datasets\Internet_Dial-BAf.csv
Guardado: datasets\Internet_Totales_Dial-BAf.csv
Guardado: datasets\Internet_Ingresos.csv
Guardado: datasets\mapa_conectividad_Hoja3.csv
Guardado: datasets\Portabilidad_Portabilidad.csv
Guardado: datasets\Portabilidad_Portin.csv
Guardado: datasets\telefonia_fija_Fija_prov_tipo.csv
Guardado: datasets\t

Avanzamos con la interpretación de los datos

In [3]:
#leer e imprimir la informacion de cada archivo csv
def print_csv_info(file_path):
    df = pd.read_csv(file_path)
    file_name = os.path.basename(file_path)
    
    print(f"Archivo: {file_name}")
    print(f"Columnas: {df.columns.tolist()}")
    print(f"Primera fila: {df.iloc[0].tolist()}")
    print("-" * 10)

#leer csv
for file in os.listdir(carpeta):
    if file.endswith(".csv"):
        file_path = os.path.join(carpeta, file)
        print_csv_info(file_path)

print("REALIZADO")


Archivo: Internet_Accesos_Por_Tecnologia.csv
Columnas: ['Año', 'Trimestre', 'Provincia', 'ADSL', 'Cablemodem', 'Fibra óptica', 'Wireless', 'Otros', 'Total']
Primera fila: ['2024', '1', 'Buenos Aires', 203297.0, 2790663.0, 1799013.0, 150290.0, 101294.0, 5044557.0]
----------
Archivo: Internet_Accesos_por_velocidad.csv
Columnas: ['Año', 'Trimestre', 'Provincia', 'HASTA 512 kbps', '+ 512 Kbps - 1 Mbps', '+ 1 Mbps - 6 Mbps', '+ 6 Mbps - 10 Mbps', '+ 10 Mbps - 20 Mbps', '+ 20 Mbps - 30 Mbps', '+ 30 Mbps', 'OTROS', 'Total']
Primera fila: [2024, 1, 'Buenos Aires', 26002.0, 22510.0, 221185.0, 233225.0, 241778.0, 106418.0, 4068292.0, 125147.0, 5044557.0]
----------
Archivo: Internet_Accesos_tecnologia_localidad.csv
Columnas: ['Provincia', 'Partido', 'Localidad', 'Link Indec', 'ADSL', 'CABLEMODEM', 'DIAL UP', 'FIBRA OPTICA', 'OTROS', 'SATELITAL', 'WIMAX', 'WIRELESS', 'Total general']
Primera fila: ['BUENOS AIRES', '25 de Mayo', '25 de Mayo', '6854100', 755, 4600, 0, 2, 0, 742, 0, 727, 6826]
----

PROCEDEMOS CON LA LIMPIEZA:

### funciones:

In [4]:
#eliminar duplicados
def eliminar_duplicados(df):
    antes = len(df)
    df = df.drop_duplicates()
    despues = len(df)
    if antes != despues:
        print(f"--se eliminaron {antes - despues} duplicados")
    return df

#remplazar nulos por 0
def manejar_nulos(df):
    nulos_antes = df.isnull().sum().sum()
    df = df.fillna(0)
    nulos_despues = df.isnull().sum().sum()
    if nulos_antes != nulos_despues:
        print(f"--se reemplazaron {nulos_antes} valores nulos")
    return df

#estandarizar
def estandarizar_columnas(df):
    columnas_antes = df.columns.tolist()
    df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')
    columnas_despues = df.columns.tolist()
    if columnas_antes != columnas_despues:
        print("--se estandarizaron las columnas")
    return df

#borrar columnas con su mayoria vacias
def borrar_columnas_vacias(df, umbral=0.8):
    columnas_antes = df.shape[1]
    umbral_filas = umbral * len(df)
    df = df.dropna(axis=1, thresh=umbral_filas).loc[:, (df == 0).sum() < umbral_filas]
    columnas_despues = df.shape[1]
    if columnas_antes != columnas_despues:
        print(f"--se eliminaron {columnas_antes - columnas_despues} columnas vacias")
    return df

#borrar ultima fila (descripciones fuera de la tabla)
def borrar_ultimas_filas_csv(ruta_csv,cantidad):
    df = pd.read_csv(ruta_csv)

    df = df[:-cantidad]

    df.to_csv(ruta_csv, index=False)

    print(f"La última fila ha sido eliminada de: {ruta_csv}")

#borrar decimales .0
def borrar_decimales_cero(df):
    for col in df.select_dtypes(include=['float64']):
        df[col] = df[col].astype(int)

    return df

#funcion para borrar elementos del dataframe
def clear_error(df):
    #contar la cantidad de datos originales
    original_count = df.size
    #borrar espacios en blanco y asteriscos
    df = df.applymap(lambda x: x.replace('*', '').strip() if isinstance(x, str) else x)
    df.replace('', pd.NA, inplace=True)
    df.dropna(inplace=True)
    #contar eliminados
    cleaned_count = df.size
    deleted_count = original_count - cleaned_count
    print(f"--se eliminaron {deleted_count} datos del df")
    return df

### Ejecución:

In [5]:
for archivo in os.listdir(carpeta):
    if archivo.endswith('.csv'):
        ruta_archivo = os.path.join(carpeta, archivo)
        
        #leer csv
        data = pd.read_csv(ruta_archivo)
        print(f'#cambios en {archivo}:')
        
        #aplicar funciones
        #data = ajustar_fechas(data)
        data = eliminar_duplicados(data)
        data = manejar_nulos(data)
        data = estandarizar_columnas(data)
        data = borrar_columnas_vacias(data)
        data = borrar_decimales_cero(data)
        data = clear_error(data)
        
        #guardar archivo limpio
        ruta_guardado = os.path.join(carpeta, f'{archivo}')
        print(f'listo {archivo}')
        data.to_csv(ruta_guardado, index=False)

borrar_ultimas_filas_csv('datasets/Internet_Accesos_Por_Tecnologia.csv',2)
borrar_ultimas_filas_csv('datasets/Portabilidad_Portin.csv',2)

#cambios en Internet_Accesos_Por_Tecnologia.csv:
--se reemplazaron 16 valores nulos
--se estandarizaron las columnas
--se eliminaron 9 datos del df
listo Internet_Accesos_Por_Tecnologia.csv
#cambios en Internet_Accesos_por_velocidad.csv:
--se reemplazaron 6 valores nulos
--se estandarizaron las columnas
--se eliminaron 0 datos del df
listo Internet_Accesos_por_velocidad.csv
#cambios en Internet_Accesos_tecnologia_localidad.csv:
--se eliminaron 279 duplicados
--se reemplazaron 4 valores nulos
--se estandarizaron las columnas
--se eliminaron 2 columnas vacias
--se eliminaron 0 datos del df
listo Internet_Accesos_tecnologia_localidad.csv
#cambios en Internet_Acc_vel_loc_sinrangos.csv:
--se reemplazaron 351902 valores nulos
--se estandarizaron las columnas
--se eliminaron 107 columnas vacias
--se eliminaron 0 datos del df
listo Internet_Acc_vel_loc_sinrangos.csv
#cambios en Internet_Dial-BAf.csv:
--se reemplazaron 2 valores nulos
--se estandarizaron las columnas
--se eliminaron 0 datos del

C:\Users\Mati\AppData\Local\Temp\ipykernel_21492\110646249.py:60: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.replace('*', '').strip() if isinstance(x, str) else x)
C:\Users\Mati\AppData\Local\Temp\ipykernel_21492\110646249.py:60: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.replace('*', '').strip() if isinstance(x, str) else x)
C:\Users\Mati\AppData\Local\Temp\ipykernel_21492\110646249.py:60: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.replace('*', '').strip() if isinstance(x, str) else x)
C:\Users\Mati\AppData\Local\Temp\ipykernel_21492\110646249.py:60: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.replace('*', '').strip() if isinstance(x, str) else x)
C:\Users\Mati\AppData\Local\Temp\ipykernel_21492\110646249.py:60

#cambios en Internet_Totales_Accesos_Por_Tecnologia.csv:
--se estandarizaron las columnas
--se eliminaron 0 datos del df
listo Internet_Totales_Accesos_Por_Tecnologia.csv
#cambios en Internet_Totales_Accesos_por_velocidad.csv:
--se estandarizaron las columnas
--se eliminaron 0 datos del df
listo Internet_Totales_Accesos_por_velocidad.csv
#cambios en Internet_Totales_Dial-BAf.csv:
--se estandarizaron las columnas
--se eliminaron 0 datos del df
listo Internet_Totales_Dial-BAf.csv
#cambios en Internet_Totales_VMD.csv:
--se estandarizaron las columnas
--se eliminaron 0 datos del df
listo Internet_Totales_VMD.csv
#cambios en Internet_Velocidad_%_por_prov.csv:
--se estandarizaron las columnas
--se eliminaron 0 datos del df
listo Internet_Velocidad_%_por_prov.csv
#cambios en Internet_Velocidad_sin_Rangos.csv:
--se reemplazaron 192 valores nulos
--se estandarizaron las columnas
--se eliminaron 154 columnas vacias
--se eliminaron 0 datos del df
listo Internet_Velocidad_sin_Rangos.csv
#cambios e

C:\Users\Mati\AppData\Local\Temp\ipykernel_21492\110646249.py:60: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.replace('*', '').strip() if isinstance(x, str) else x)
C:\Users\Mati\AppData\Local\Temp\ipykernel_21492\110646249.py:60: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.replace('*', '').strip() if isinstance(x, str) else x)
C:\Users\Mati\AppData\Local\Temp\ipykernel_21492\110646249.py:60: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.replace('*', '').strip() if isinstance(x, str) else x)
C:\Users\Mati\AppData\Local\Temp\ipykernel_21492\110646249.py:60: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.replace('*', '').strip() if isinstance(x, str) else x)
C:\Users\Mati\AppData\Local\Temp\ipykernel_21492\110646249.py:60

#cambios en Portabilidad_Portabilidad.csv:
--se estandarizaron las columnas
--se eliminaron 1 columnas vacias
--se eliminaron 0 datos del df
listo Portabilidad_Portabilidad.csv
#cambios en Portabilidad_Portin.csv:
--se reemplazaron 187 valores nulos
--se estandarizaron las columnas
--se eliminaron 1 columnas vacias
--se eliminaron 0 datos del df
listo Portabilidad_Portin.csv
#cambios en telefonia_fija_Fija_accesos_tot.csv:
--se estandarizaron las columnas
--se eliminaron 0 datos del df
listo telefonia_fija_Fija_accesos_tot.csv
#cambios en telefonia_fija_Fija_ingresos.csv:
--se estandarizaron las columnas
--se eliminaron 0 datos del df
listo telefonia_fija_Fija_ingresos.csv
#cambios en telefonia_fija_Fija_penetracion_prov.csv:
--se estandarizaron las columnas
--se eliminaron 0 datos del df
listo telefonia_fija_Fija_penetracion_prov.csv
#cambios en telefonia_fija_Fija_penetracion_tot.csv:
--se estandarizaron las columnas
--se eliminaron 0 datos del df
listo telefonia_fija_Fija_penetracio

C:\Users\Mati\AppData\Local\Temp\ipykernel_21492\110646249.py:60: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.replace('*', '').strip() if isinstance(x, str) else x)
C:\Users\Mati\AppData\Local\Temp\ipykernel_21492\110646249.py:60: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.replace('*', '').strip() if isinstance(x, str) else x)
C:\Users\Mati\AppData\Local\Temp\ipykernel_21492\110646249.py:60: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.replace('*', '').strip() if isinstance(x, str) else x)
C:\Users\Mati\AppData\Local\Temp\ipykernel_21492\110646249.py:60: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.replace('*', '').strip() if isinstance(x, str) else x)
C:\Users\Mati\AppData\Local\Temp\ipykernel_21492\110646249.py:60